In [2]:
import pandas as pd
import numpy as np

#reading data
data_train = pd.read_csv('Data/train.csv')
data_test = pd.read_csv('Data/test.csv')

In [3]:
#filling null values
data_cleaner = [data_train,data_test]
for data in data_cleaner:
    data.drop(['Cabin','Ticket','PassengerId'],axis=1,inplace=True)
    data['Age'].fillna(data['Age'].median(),inplace=True)
    data['Fare'].fillna(data['Fare'].median(),inplace=True)
    data['Embarked'].fillna(data['Embarked'].mode()[0],inplace=True)
    
    

In [4]:
#create new feature
for data in data_cleaner:
    data['Title'] = data['Name'].apply(lambda x : x.split(',')[1].split('.')[0])
    min_title = data['Title'].value_counts() < 10
    data['Title'] = data['Title'].apply(lambda x: 'Misc' if min_title.loc[x] == True else x)
    data['AgeBin'] = pd.cut(data['Age'].astype(int),5)
    data['FareBin'] = pd.qcut(data['Fare'].astype(int),4)
    data['Family_size'] = data['SibSp'] + data['Parch'] + 1
    data['Is_Alone'] = 1
    data['Is_Alone'].loc[data['Family_size'] > 1] = 0

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
from sklearn.preprocessing import LabelEncoder

#encode data
encoder = LabelEncoder()
for data in data_cleaner:
    data['Sex_Code'] = encoder.fit_transform(data['Sex'])
    data['Age_Code'] = encoder.fit_transform(data['AgeBin'])
    data['Fare_Code'] = encoder.fit_transform(data['FareBin'])
    data['Embarked_Code'] = encoder.fit_transform(data['Embarked'])
    data['Title_Code'] = encoder.fit_transform(data['Title'])

In [6]:
data_train_x = ['Age','Pclass','Sex','Family_size','Title','FareBin','Embarked']
data_train_x_calc = ['Pclass','Family_size','Is_Alone','Sex_Code','Age_Code','Fare_Code','Embarked_Code','Title_Code']
Target = ['Survived']

In [7]:
for x in data_train_x:
    if data_train[x].dtype != 'float64' :
        print('Survival Correlation by:', x)
        print(data_train[[x, Target[0]]].groupby(x, as_index=False).mean())
        print('-'*10, '\n')

Survival Correlation by: Pclass
   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363
---------- 

Survival Correlation by: Sex
      Sex  Survived
0  female  0.742038
1    male  0.188908
---------- 

Survival Correlation by: Family_size
   Family_size  Survived
0            1  0.303538
1            2  0.552795
2            3  0.578431
3            4  0.724138
4            5  0.200000
5            6  0.136364
6            7  0.333333
7            8  0.000000
8           11  0.000000
---------- 

Survival Correlation by: Title
     Title  Survived
0   Master  0.575000
1     Miss  0.697802
2       Mr  0.156673
3      Mrs  0.792000
4     Misc  0.444444
---------- 

Survival Correlation by: FareBin
         FareBin  Survived
0  (-0.001, 7.0]  0.215768
1    (7.0, 14.0]  0.287037
2   (14.0, 31.0]  0.457399
3  (31.0, 512.0]  0.597156
---------- 

Survival Correlation by: Embarked
  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009
----

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

#Create Correlation df
corr = data_train.corr()
#Plot figsize
fig, ax = plt.subplots(figsize=(15, 15))
#Generate Color Map
colormap = sns.diverging_palette(220, 10, as_cmap=True)
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 })
#show plot
plt.show()

<Figure size 1500x1500 with 2 Axes>

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection

cv_split = model_selection.ShuffleSplit(n_splits=10, test_size=0.3, train_size=0.6, random_state=0)

dtree = DecisionTreeClassifier(random_state=0)
base_results = model_selection.cross_validate(dtree, data_train[data_train_x_calc], data_train[Target], cv  = cv_split,return_train_score=True)
dtree.fit(data_train[data_train_x_calc], data_train[Target])

print('BEFORE DT Parameters: ', dtree.get_params())
print("BEFORE DT Training w/bin score mean: {:.2f}". format(base_results['train_score'].mean()*100))
print("BEFORE DT Test w/bin score mean: {:.2f}". format(base_results['test_score'].mean()*100))
print("BEFORE DT Test w/bin score 3*std: +/- {:.2f}". format(base_results['test_score'].std()*100*3))
print('-'*10)

dtree_no_param = DecisionTreeClassifier()

param_grid = {'criterion': ['gini', 'entropy'],  #scoring methodology; two supported formulas for calculating information gain - default is gini
              #'splitter': ['best', 'random'], #splitting methodology; two supported strategies - default is best
              'max_depth': [2,4,6,8,10,None], #max depth tree can grow; default is none
              #'min_samples_split': [2,5,10,.03,.05], #minimum subset size BEFORE new split (fraction is % of total); default is 2
              #'min_samples_leaf': [1,5,10,.03,.05], #minimum subset size AFTER new split split (fraction is % of total); default is 1
              #'max_features': [None, 'auto'], #max features to consider when performing split; default none or all
              'random_state': [0] #seed or control random number generator: https://www.quora.com/What-is-seed-in-random-number-generation
             }

tune_model = model_selection.GridSearchCV(dtree_no_param,param_grid=param_grid,scoring='roc_auc',cv = cv_split,return_train_score=True)
tune_model.fit(data_train[data_train_x_calc],data_train[Target])

print('AFTER DT Parameters: ', tune_model.best_params_)
#print(tune_model.cv_results_['mean_train_score'])
print("AFTER DT Training w/bin score mean: {:.2f}". format(tune_model.cv_results_['mean_train_score'][tune_model.best_index_]*100)) 
#print(tune_model.cv_results_['mean_test_score'])
print("AFTER DT Test w/bin score mean: {:.2f}". format(tune_model.cv_results_['mean_test_score'][tune_model.best_index_]*100))
print("AFTER DT Test w/bin score 3*std: +/- {:.2f}". format(tune_model.cv_results_['std_test_score'][tune_model.best_index_]*100*3))
print('-'*10)

BEFORE DT Parameters:  {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': 0, 'splitter': 'best'}
BEFORE DT Training w/bin score mean: 89.57
BEFORE DT Test w/bin score mean: 81.98
BEFORE DT Test w/bin score 3*std: +/- 6.17
----------
AFTER DT Parameters:  {'criterion': 'entropy', 'max_depth': 4, 'random_state': 0}
AFTER DT Training w/bin score mean: 89.28
AFTER DT Test w/bin score mean: 87.45
AFTER DT Test w/bin score 3*std: +/- 6.14
----------


In [10]:
from sklearn import feature_selection

print('BEFORE RFE : ', dtree.get_params())
print("BEFORE RFE Training w/bin score mean: {:.2f}". format(base_results['train_score'].mean()*100))
print("BEFORE RFE Test w/bin score mean: {:.2f}". format(base_results['test_score'].mean()*100))
print("BEFORE RFE Test w/bin score 3*std: +/- {:.2f}". format(base_results['test_score'].std()*100*3))
print('-'*10)

dtree_rfe = feature_selection.RFECV(dtree, step = 1, scoring = 'accuracy', cv = cv_split)
dtree_rfe.fit(data_train[data_train_x_calc], data_train[Target])

X_rfe = data_train[data_train_x_calc].columns.values[dtree_rfe.get_support()]
rfe_results = model_selection.cross_validate(dtree, data_train[X_rfe], data_train[Target], cv  = cv_split,return_train_score=True)

print('AFTER DT RFE Training Shape New: ', data_train[X_rfe].shape) 
print('AFTER DT RFE Training Columns New: ', X_rfe)

print("AFTER DT RFE Training w/bin score mean: {:.2f}". format(rfe_results['train_score'].mean()*100)) 
print("AFTER DT RFE Test w/bin score mean: {:.2f}". format(rfe_results['test_score'].mean()*100))
print("AFTER DT RFE Test w/bin score 3*std: +/- {:.2f}". format(rfe_results['test_score'].std()*100*3))
print('-'*10)

best_model = model_selection.GridSearchCV(dtree_no_param,param_grid=param_grid,scoring='roc_auc',cv = cv_split,return_train_score=True)
best_model.fit(data_train[X_rfe],data_train[Target])

print('AFTER DT RFE Tuned Parameters: ', best_model.best_params_)
#print(rfe_tune_model.cv_results_['mean_train_score'])
print("AFTER DT RFE Tuned Training w/bin score mean: {:.2f}". format(best_model.cv_results_['mean_train_score'][best_model.best_index_]*100)) 
#print(rfe_tune_model.cv_results_['mean_test_score'])
print("AFTER DT RFE Tuned Test w/bin score mean: {:.2f}". format(best_model.cv_results_['mean_test_score'][best_model.best_index_]*100))
print("AFTER DT RFE Tuned Test w/bin score 3*std: +/- {:.2f}". format(best_model.cv_results_['std_test_score'][best_model.best_index_]*100*3))
print('-'*10)

BEFORE RFE :  {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': 0, 'splitter': 'best'}
BEFORE RFE Training w/bin score mean: 89.57
BEFORE RFE Test w/bin score mean: 81.98
BEFORE RFE Test w/bin score 3*std: +/- 6.17
----------
AFTER DT RFE Training Shape New:  (891, 6)
AFTER DT RFE Training Columns New:  ['Pclass' 'Family_size' 'Sex_Code' 'Age_Code' 'Fare_Code' 'Title_Code']
AFTER DT RFE Training w/bin score mean: 88.30
AFTER DT RFE Test w/bin score mean: 82.65
AFTER DT RFE Test w/bin score 3*std: +/- 5.76
----------


C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AFTER DT RFE Tuned Parameters:  {'criterion': 'gini', 'max_depth': 4, 'random_state': 0}
AFTER DT RFE Tuned Training w/bin score mean: 89.37
AFTER DT RFE Tuned Test w/bin score mean: 87.26
AFTER DT RFE Tuned Test w/bin score 3*std: +/- 6.58
----------


In [27]:
test = data_test.drop(['Name','Sex','Age','SibSp','Parch','Embarked','Title','AgeBin','FareBin','Embarked_Code','Fare','Is_Alone'],axis=1)
result_model = DecisionTreeClassifier(criterion='gini',max_depth=4,random_state=0)
result_model.fit(data_train[X_rfe],data_train[Target])
Y_pred = result_model.predict(test)

In [28]:
blas = pd.read_csv('Data/test.csv')
submission = pd.DataFrame({
        "PassengerId": blas["PassengerId"],
        "Survived": Y_pred
    })

In [31]:
submission.to_csv('submission.csv', index=False)